# SITCOMTN-083 - Force Actuator Following Error Lag Analysis

As described in [SITCOM-1167], we are seeing delays in the Force Actuators applied forces when comparing them with the demanded forces.  
Now, we want to quantify those delays and have statistics on all the actuators.  
  
The [M1M3 Actuator Forces dashboard in USDF] contains the forces we want to use for this analysis.  
The url for this dashboard is created using the [m1m3_bump_test_times.py] Python script.  

The easiest way to do this is with the bump test results, since here there is a well defined applied force.  
This is why this notebook is associated with the technote [SITCOMTN-083].  
The notebook works by taking the applied force and the measured force, then finding the delay which minimizes the difference between these two forces.

[SITCOM-1167]: https://jira.lsstcorp.org/browse/SITCOM-1167
[SITCOMTN-083]: https://sitcomtn-083.lsst.io/
[M1M3 Actuator Forces dashboard in USDF]: https://usdf-rsp.slac.stanford.edu/chronograf/sources/1/dashboards/61?refresh=Paused&tempVars%5Bz_index%5D=112&tempVars%5By_index%5D=0&tempVars%5Bx_index%5D=112&tempVars%5Bs_index%5D=112&lower=now%28%29%20-%205m
[m1m3_bump_test_times.py]: https://github.com/lsst-ts/ts_criopy/blob/develop/python/lsst/ts/criopy/m1m3_bump_tests_times.py

This notebook and the associated Python code has been modified so that it now can calculate the delays from either bump tests or from actual slews of the TMA

## Notebook Preparations

Let's have here all the imports and global variables we will need during the notebook execution.  

In [ ]:
# Initial time to find out executions of bump tests
start_time = "2024-01-04T10:00:00"

# End time to find out executions of bump tests
end_time = "2024-01-04T11:15:00"

# "Gentle" slew event
gentle = [20240105, 1047, 320]

# "Aggressive" slew event
aggressive = [20240102, 607, 108]

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import asyncio
import glob
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

from astropy.time import Time, TimeDelta
from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages
from pathlib import Path
from scipy.optimize import minimize

from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker
from lsst.sitcom.vandv.m1m3 import actuator_analysis as act
from lsst.ts.xml.tables.m1m3 import FATable

In [ ]:
# Create an EFD client for queries in the notebook
efd_client = makeEfdClient()
eventMaker = TMAEventMaker()
# Create a folder to store plots
plot_dir = Path("./plots")
plot_dir.mkdir(exist_ok=True, parents=True)

# First, analyze delays from bump tests.

## Single Actuator Analysis

In [ ]:
bump_test_status = getEfdData(
    client=efd_client,
    topic="lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus",
    columns="*",
    begin=Time(start_time, scale="utc", format="isot"),
    end=Time(end_time, scale="utc", format="isot"),
)

In [ ]:
%matplotlib inline

# The actuator fa_id runs from 101 to 443, as described in
# Section 2 of https://sitcomtn-083.lsst.io/
force_actuator_id = 212

# Grab the first bump test within the date range
bump_test_index = 0

fig = plt.figure(figsize=(10, 10))

[delay_primary, delay_secondary] = act.plot_actuator_delay(
    fig, efd_client, force_actuator_id, bt_results=bump_test_status
)

plt.savefig(str(plot_dir / f"Bump_Test_Delays_{force_actuator_id}.png"))

## Analyze all the Force actuators

In [ ]:
# Allocate space to store the data
delay_dict = {}

# Create a PDF file to fill with data
timestamp = (
    bump_test_status.index[0]
    .isoformat()
    .split(".")[0]
    .replace("-", "")
    .replace(":", "")
)
pdf = PdfPages(str(plot_dir / f"Bump_Test_Delays_{timestamp}.pdf"))

# Create a figure
fig = plt.figure(figsize=(10, 10))

# Loop over all the force actuators
for index in range(len(FATable)):
    # try:
    # Extract actuator data
    id = FATable[index].actuator_id
    [primary_delay, secondary_delay] = act.plot_actuator_delay(
        fig, efd_client, id, bt_results=bump_test_status
    )
    print(f"Plot for actuator {id} succeeded!")

    # Saves the current figure into a pdf page
    pdf.savefig(fig)

    # Clear figure
    plt.clf()

    # Store delays
    delay_dict[id] = [primary_delay, secondary_delay]

    ## Todo: find a more specific exeption.
    ## @b1quint - I will leave this commented out until we find the proper error
    # except:
    #     print(f"Plot for actuator {id} failed!")
    #     delay_dict[id] = [None, None]
    #     continue
pdf.close()
print(f"Plots completed and PDF closed")

## Delay Histograms

In [ ]:
# Create placeholders
primary_delays = []
secondary_delays = []

# Strip primary and secondary delays from the dictionary above
for index in range(len(FATable)):
    id = FATable[index].actuator_id

    try:
        [primary_delay, secondary_delay] = delay_dict[id]
    except KeyError:
        print("Empty dictionary. Make sure you run previous cell.")
    except NameError:
        print("Delay dictionary not defined. Make sure you run previous cell.")

    primary_delays.append(primary_delay)
    secondary_delays.append(secondary_delay)

# Convert lists into arrays
primary_delays = np.array(primary_delays, dtype=float)
secondary_delays = np.array(secondary_delays, dtype=float)

# Remove NaNs
primary_delays = primary_delays[~np.isnan(primary_delays)]
secondary_delays = secondary_delays[~np.isnan(secondary_delays)]

# Remove outliers
primary_delays = primary_delays[primary_delays < 500]
secondary_delays = secondary_delays[secondary_delays < 500]

# Extract timestamp
timestamp = bump_test_status.index[0].isoformat().split(".")[0]

# Create histograms and save them
fig = plt.figure(figsize=(10, 5))
act.plot_delay_histograms(fig, timestamp, primary_delays, secondary_delays)
plt.savefig(
    str(
        plot_dir
        / f"Bump_Test_Delay_Histograms_{timestamp.replace('-', '').replace(':', '')}.png"
    )
)

# Next, analyze delays from actual slews.

## Start with a "gentle" slew, with these parameters:
Azimuth Velocity - 4.0 degrees/sec

Azimuth acceleration - 4.0 degrees/sec^2

Azimuth jerk - 2.0 degrees/sec^3

Elevation Velocity - 2.0 degrees/sec

Elevation acceleration - 2.0 degrees/sec^2

Elevation jerk - 1.0 degrees/sec^3

In [ ]:
%matplotlib inline

[dayObs, seqNum, force_actuator_id] = gentle
event = eventMaker.getEvent(dayObs, seqNum)

fig = plt.figure(figsize=(10, 10))

[delay_primary, delay_secondary] = act.plot_actuator_delay(
    fig, efd_client, force_actuator_id, event=event
)

plt.savefig(str(plot_dir / f"Event_{dayObs}-{seqNum}_Delays_{force_actuator_id}.png"))

## Now a more aggressive slew, with these parameters:
Azimuth Velocity - 7.0 degrees/sec

Azimuth acceleration - 7.0 degrees/sec^2

Azimuth jerk - 28.0 degrees/sec^3

Elevation Velocity - 3.5 degrees/sec

Elevation acceleration - 3.5 degrees/sec^2

Elevation jerk - 14.0 degrees/sec^3

## In this case the measured forces look nothing like the applied forces, so the calculated delay is suspect.

In [ ]:
%matplotlib inline

[dayObs, seqNum, force_actuator_id] = aggressive
event = eventMaker.getEvent(dayObs, seqNum)

fig = plt.figure(figsize=(10, 10))

[delay_primary, delay_secondary] = act.plot_actuator_delay(
    fig, efd_client, force_actuator_id, event=event
)

plt.savefig(str(plot_dir / f"Event {dayObs}-{seqNum}_Delays_{force_actuator_id}.png"))